In [22]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [23]:
load_dotenv()
api_key= os.getenv('OPENAI_API_KEY')


In [24]:
if not api_key:
    print("NO Key")
elif api_key[:8]!="sk-proj-":
    print("No Start KEY")
elif api_key.strip()!=api_key:
    print("Remove spaces")
else :
    print("OK key")


OK key


In [25]:
MODEL='gpt-4o-mini'
openai= OpenAI()

In [26]:
class Website:
    """
    A utility class to represent a website that we scraped
    """
    url: str
    title: str
    text: str
    def __init__(self,url):
        """
        Create this Website object from f=given url using BeautifulSoup library
        """
        self.url= url
        response = requests.get(url)
        soup= BeautifulSoup(response.content,'html.parser')
        self.title= soup.title.string if soup.title else "No Title Found!"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text= soup.body.get_text(separator="\n", strip=True)
        else:
            self.text= "No Body Found!"
        links=[link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
            
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Cpntents:\n{self.text}\n\n"

In [27]:
ed=Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

In [28]:
link_system_prompt= "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links" :[
        {"type": "about page","url":"https://full.url/goes/here/about"}
        {"type": "careers page","url":"https://another.full.url/careers"}
    ]
}
"""

In [29]:
link_system_prompt

'You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.\nYou should respond in JSON as in this example:\n{\n    "links" :[\n        {"type": "about page","url":"https://full.url/goes/here/about"}\n        {"type": "careers page","url":"https://another.full.url/careers"}\n    ]\n}\n'

In [30]:
def get_links_user_prompt_for(website):
    user_prompt= f"Here is the list of links on the website of {website.title} -"
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [31]:
print(get_links_user_prompt_for(ed))

Here is the list of links on the website of Home - Edward Donner -please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.

In [32]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt_for(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [34]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'inference/get-started',
 '/spaces',
 '/models',
 '/zai-org/GLM-4.5',
 '/Qwen/Qwen3-Coder-480B-A35B-Instruct',
 '/tencent/HunyuanWorld-1',
 '/bosonai/higgs-audio-v2-generation-3B-base',
 '/Qwen/Qwen3-235B-A22B-Thinking-2507',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zumjoy/Multi-Style_Video-to-Anime_Generator',
 '/spaces/smola/higgs_audio_v2',
 '/spaces/Qwen/Qwen3-Coder-WebDev',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/NousResearch/Hermes-3-Dataset',
 '/datasets/interstellarninja/hermes_reasoning_tool_use',
 '/datasets/MegaScience/MegaScience',
 '/datasets/microsoft/rStar-Coder',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/a

In [ ]:
get_links("https://huggingface.co")

# get all links and text from any given site and then generate a brochure
 *After getting the relavent links from the site we will now go through all pages of the website and get the links and body texts of the whole site*

 ## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [ ]:
def get_all_details(url):
    result="Landing Page:\n"
    result += Website(url).get_contents()
    link = get_links(url)
    for link in links["links"]:
        result +=f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [ ]:
system_prompt= system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [ ]:
def get_brochure_user_prompt(company_name,url):
    user_prompt=f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt
    

In [ ]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

In [35]:
# create the brochure finally
def create_brochure(company_name,url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name,url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

# Adding stream effect

In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    response =""
    display_handle = display(Markdown(""),display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

## Bill needs to be paid

In [ ]:

def summarize(url):
    website= Website(url)
    response= openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://edwarddonner.com")

In [ ]:
def display_summary(url):
    summary=summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://edwarddonner.com")